In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import os
from Bio import SeqIO
from itertools import combinations
import glob

orf_table = 'Ruminococcus.orfs.csv'
pa_table = 'master_presence_absence.csv'
genome_location = '../../../Data/Genomes/ruminococcus/*.fasta'

# Dictionary of average core genome divergence for each population
# calculated from MSA, available in phybreak directory
pi_dict = {0.1: 0.0138272139869,
           0.0: 0.00521122600799,
           0.2: 0.02343396299783631}

# Map of populations
pops = pd.read_table('../../../Data/Clusters/rumino_0.000355362.txt.cluster.tab.txt', index_col=0)

# Master table of all orfs
df_seq = pd.read_csv(orf_table, index_col = 'protein_accession')

# Master table of presence/absence
df_presence_absenence = pd.read_csv(pa_table, index_col='member')['cluster']
df_concat = df_seq.join(df_presence_absenence)

# Map contigs to populations
pop_map = pd.DataFrame(columns=['contig', 'population', 'strain'])
for f in glob.glob(genome_location):
    strain = '.'.join(os.path.basename(f).split('.')[0:-1])
    population = pops.loc[strain, 'Cluster_ID']
    for s in SeqIO.parse(f, 'fasta'):
        temp = pd.DataFrame([[s.id,
                              population,
                              strain]], columns=pop_map.columns)
        pop_map = pop_map.append(temp)
pop_map.index = pop_map.contig

In [4]:
# Methods for calculating pi and alignments
def align_seqs(unaligned_seqs):
    
    # Align sequences with muscle
    
    with open('temp_unalign.fasta', 'w') as outfile:
        for i, s in enumerate(unaligned_seqs):
            outfile.write('>{0}\n'.format(str(i)))
            outfile.write(s + '\n')
    os.system('muscle.exe -in temp_unalign.fasta -out temp_align.fasta')
    seqs = [str(s.seq) for s in SeqIO.parse('temp_align.fasta', 'fasta')]
    return seqs

def is_recent(aligned_seqs, alpha, pi):
    # Given aligned seqs, a significance level, and an average nucleotide divergence
    # determines whether gene is "recent"

    pi_obs = calculate_pi(aligned_seqs)
    seq_len = len(aligned_seqs[0])
    low, high = stats.binom.interval(alpha, seq_len, pi)
    if pi_obs == 0 or pi_obs < low / seq_len:
        recent = True
    else:
        recent = False
    return recent
    
def count_divs(s1, s2):
    # just counts divergences
    d = 0
    for b1, b2 in zip(s1, s2):
        if b1 != b2:
            d += 1
    return d * 1.0 / len(s1)

def calculate_pi(aligned_seqs):
    # Calculates average nucleotide divergence
    aves = []
    for seq1, seq2 in combinations(aligned_seqs, 2):
        aves.append(count_divs(seq1, seq2))
    return(np.average(aves))
    

In [3]:
# fix df_concat fasta ids
df_concat['fasta_id'] = ['_'.join(i.split('_')[0:-1]) for i in df_concat.index]
# assign population to contigs
for index, row in df_concat.iterrows():
    contig = row.fasta_id
    population = pop_map.loc[contig, 'population']
    df_concat.loc[index, 'population'] = population

In [17]:
# Process annotations
dfs = []
colnames = ("protein_accession",
            "md5",
            "sequence_length",
            "analysis",
            "signature_accession",
            "signature_description",
            "start",
            "stop",
            "score",
            "status",
            "date",
            "interpro_accession",
            "interpro_description",
            "go_annotation",
            "pathway_annotation")

for f in glob.glob('annotations/*.tsv'):
    newlines = [line for line in open(f) if len(line.split('\t')) == 15]
    with open(f + '.filtered.tsv', 'w') as outfile:
        outfile.writelines(newlines)
for f in glob.glob('annotations/*filtered.tsv'):
    temp = pd.read_table(f, header=None, names=colnames)
    dfs.append(temp)
    
annotations = pd.concat(dfs)

In [28]:
finaldf = pd.DataFrame(index=set(df_concat.cluster), columns=['Population', 'Annotation'])
for prot_cluster, prot_df in df_concat.groupby('cluster'):
    num_pops = len(set(prot_df.population))
    num_strains = len(set(prot_df.fasta_id))
    # Checks if the orf cluster only contains one population
    if num_pops == 1:
        pop = list(set(prot_df.population))[0]
        pop_length = len(set(pops[pops.Cluster_ID == pop].Clonal_complex))
        # Checks if all members of the population have this protein
        if num_strains == pop_length:
            aligned_seqs = align_seqs(prot_df.orf_seq)
            
            # Check nucleotide diversity cutoff
            if is_recent(aligned_seqs, 0.05, pi_dict[pop]):
                annots = annotations[annotations.protein_accession == prot_cluster]
                finaldf.loc[prot_cluster, 'Population'] = pop
                if set(annots.interpro_description) != set():
                    finaldf.loc[prot_cluster, 'Annotation'] = '; '.join(set(annots.interpro_description))
                else:
                    finaldf.loc[prot_cluster, 'Annotation'] = 'Unannotated'

finaldf = finaldf.dropna()
finaldf = finaldf.sort_values('Population')
finaldf.to_csv('flex_genome_results.csv')